In [48]:
from kubernetes import client, config
# Configs can be set in Configuration class directly or using helper utility
config.load_kube_config()
apiCoreV1 = client.CoreV1Api()
apiAppsV1 = client.AppsV1Api() #
apiCustObject = client.CustomObjectsApi() # Metrics


In [139]:
def get_metrics_by_deployment(deployment):
  dname = deployment.metadata.labels['app']
  kwargs = dict(
    group="metrics.k8s.io",version="v1beta1", 
    namespace="default", plural="pods",
    label_selector=f'app={dname}'
  )
  resource = apiCustObject.list_namespaced_custom_object(**kwargs)
  return resource

In [121]:
def get_pods_by_deployment(deployment):
  # name like `details`
  dname = deployment.metadata.labels['app']
  # apiCoreV1.list_namespaced_pod(namespace='default', label_selector=f'app=details')
  return apiCoreV1.list_namespaced_pod(namespace='default', label_selector=f'app={dname}')

In [125]:
deployments = apiAppsV1.list_namespaced_deployment('default')
dname = deployments.items[0].metadata.name
dnamespace = deployments.items[0].metadata.namespace
replicas = deployments.items[0].spec.replicas

In [126]:
pod_list = get_pods_by_deployment(deployments.items[0])

In [140]:
get_metrics_by_deployment(deployments.items[0])

{'kind': 'PodMetricsList',
 'apiVersion': 'metrics.k8s.io/v1beta1',
 'metadata': {},
 'items': [{'metadata': {'name': 'details-v1-5498c86cf5-4wxd7',
    'namespace': 'default',
    'creationTimestamp': '2022-05-16T10:20:19Z',
    'labels': {'app': 'details',
     'pod-template-hash': '5498c86cf5',
     'security.istio.io/tlsMode': 'istio',
     'service.istio.io/canonical-name': 'details',
     'service.istio.io/canonical-revision': 'v1',
     'version': 'v1'}},
   'timestamp': '2022-05-16T10:20:11Z',
   'window': '13.666s',
   'containers': [{'name': 'istio-proxy',
     'usage': {'cpu': '2997265n', 'memory': '35996Ki'}},
    {'name': 'details', 'usage': {'cpu': '0', 'memory': '14748Ki'}}]}]}

In [138]:
kwargs = dict(
  group="metrics.k8s.io",version="v1beta1", 
  namespace="default", plural="pods",
  label_selector=f'app=details'
)
resource = apiCustObject.list_namespaced_custom_object(**kwargs)


In [120]:
for deployment in deployments.items:
  if hasattr(deployment.metadata, 'labels') and deployment.metadata.labels:
    name = deployment.metadata.labels['app']
    print(name)
    print(get_pods_by_deployment(deployment))

details
details
{'api_version': 'v1',
 'items': [{'api_version': None,
            'kind': None,
            'metadata': {'annotations': {'kubectl.kubernetes.io/default-container': 'details',
                                         'kubectl.kubernetes.io/default-logs-container': 'details',
                                         'prometheus.io/path': '/stats/prometheus',
                                         'prometheus.io/port': '15020',
                                         'prometheus.io/scrape': 'true',
                                         'sidecar.istio.io/status': '{"initContainers":["istio-init"],"containers":["istio-proxy"],"volumes":["istio-envoy","istio-data","istio-podinfo","istio-token","istiod-ca-cert"],"imagePullSecrets":null,"revision":"default"}'},
                         'cluster_name': None,
                         'creation_timestamp': datetime.datetime(2022, 5, 10, 13, 39, 23, tzinfo=tzlocal()),
                         'deletion_grace_period_seconds':

In [59]:
resource['items'][1]

{'metadata': {'name': 'details-v1-5498c86cf5-4wxd7',
  'namespace': 'default',
  'creationTimestamp': '2022-05-16T09:27:29Z',
  'labels': {'app': 'details',
   'pod-template-hash': '5498c86cf5',
   'security.istio.io/tlsMode': 'istio',
   'service.istio.io/canonical-name': 'details',
   'service.istio.io/canonical-revision': 'v1',
   'version': 'v1'}},
 'timestamp': '2022-05-16T09:27:11Z',
 'window': '16.094s',
 'containers': [{'name': 'istio-proxy',
   'usage': {'cpu': '2148589n', 'memory': '35908Ki'}},
  {'name': 'details', 'usage': {'cpu': '0', 'memory': '14748Ki'}}]}

In [136]:
# Print pod name
for pod in pod_list.items:
  print(pod.metadata.name)

details-v1-5498c86cf5-4wxd7


'details-v1'